In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def create_series_w_broadening(x_values, y_values, x_arr, fwhm, shape='g'):
    spectrum = np.zeros(len(x_arr))
    def lorentzian(x_):
        #factor = np.pi*fwhm/2 # to make maximum 1.0
        return 0.5*fwhm/(np.pi*(x_**2+(0.5*fwhm)**2))
    def gaussian(x_):
        sigma = fwhm/2.3548
        return 1/(sigma*np.sqrt(2*np.pi))*np.exp(-x_**2/(2*sigma**2))
    for xv, yv in zip(x_values, y_values):
        if shape == 'g':
            spectrum += yv*gaussian(x_arr - xv)
        else:
            spectrum += yv*lorentzian(x_arr - xv)
    return spectrum

In [ ]:
def create_the_plot(fwhm, elim, over_mat, en, gas_en, gas_i_homo):
    
    de = np.min([fwhm/5, 0.005])
    energy_arr = np.arange(elim[0], elim[1], de)
    
    plt.figure(figsize=(12, 6))
    
    ### -----------------------------------------------
    ### overlap part
    
    ax2 = plt.gca()
    
    max_overlap = 0.0

    for i_gas in range(over_mat.shape[1]):
        wrt_h = i_gas - gas_i_homo
        if wrt_h < 0:
            label = "HOMO%d"%wrt_h
        elif wrt_h == 0:
            label = "HOMO"
        elif wrt_h == 1:
            label = "LUMO"
        else:
            label = "LUMO+%d"%(wrt_h-1)
        label += " (%.2f)"%gas_en[i_gas]
        series = create_series_w_broadening(en, over_mat[:, i_gas], energy_arr, fwhm)
        
        # Normalize the series such that 1 corresponds to perfect match
        gaussian_peak = 1/(fwhm/2.3548*np.sqrt(2*np.pi))
        series /= gaussian_peak
        
        if np.max(series) > max_overlap:
            max_overlap = np.max(series)
        ax2.plot(energy_arr, series, label=label, lw=2.0)
    
    overlap_lim = np.min([np.around(max_overlap+0.055, 1), 1.0])
    ax2.set_ylim([0.0, overlap_lim])
    ax2.set_ylabel("Relative projection density")
    ax2.legend(loc='upper right')

    ax2.set_xlabel("$E-E_F$ [eV]")
    plt.xlim(elim)

    plt.show()

In [ ]:
npz_path = "/home/kristjan/sshfs/daint_project/MISC/stm-overlap-tests/overlap-test-z4/overlap/overlap.npz"
loaded_data = np.load(npz_path)

In [ ]:
loaded_data.files

In [ ]:
overlap_matrix = loaded_data['overlap_matrix']
en_grp1 = loaded_data['en_grp1']
en_grp2 = loaded_data['en_grp2']
homo_grp1 = loaded_data['homo_grp1']

np.max(overlap_matrix)

In [ ]:
create_the_plot(0.05, (-2.0, 2.0), overlap_matrix, en_grp1, en_grp2, homo_grp1)